In [17]:
!pip install folium  
!pip install pandas

In [18]:
import folium 
import pandas as pd

In [19]:
from folium.plugins import MarkerCluster 
from folium.plugins import MousePosition 
from folium.features import DivIcon

In [20]:
# Task 1: Mark all launch sites on a map
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(URL)
spacex_df.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [21]:
# Create a new dataframe with only LaunchSite, Longitude, and Latitude
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first() 
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Display the new dataframe
display(launch_sites_df.head())

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [22]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [23]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name 
circle = folium.Circle(nasa_coordinate, radius=1000, color='d#35400', fill=True).add_child(folium.Popup('MASA Johnson Space Center'))
# Create a blue circile at NASA Johnson Space Center's coordinate with a icon showing its name 
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label 
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

In [24]:
#Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for lat, lng, label in zip(launch_sites_df['Lat'], launch_sites_df['Long'], launch_sites_df['Launch Site']):
    coordinate = [lat, lng]
    circle = folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(label))
    marker = folium.map.Marker(
         coordinate,
         icon=DivIcon(
             icon_size=(20,20),
             icon_anchor=(0,0),
             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label,
         )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)
display(site_map)

In [25]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [26]:
marker_cluster = MarkerCluster()

In [27]:
# Apply a function to check the value of `class` column 
# If class=1, marker_color value will be green 
# If class=0, marker_color value will be red 
def assign_marker_color(launch_class):
    if launch_class == 1:
        return 'green'
    else:
        return 'red'

# Create a new column in spacex_df dataframe called marker_color to store the marker colors based on the class value
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Display the first few rows to verify the new column
display(spacex_df.head())


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [28]:
# Add marker_cluster to current site_map 
site_map.add_child(marker_cluster) 

# for each row in spacex_df data frame 
# create a Marker object with its coordinate 
# and customize the Markers icon property to indicate if this launch was success or failed,
#e.g , icon=folium.Icon(color='white', icon_color=row['marker_color'])
for index, record in spacex_df.iterrows():
  # TODO: Create and add a Marker cluster to the site map
  coordinate = [record['Lat'], record['Long']]
  marker = folium.Marker(
      location=coordinate,
      icon=folium.Icon(color='white', icon_color=record['marker_color']),
  )
marker_cluster.add_child(marker)
site_map

In [29]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map 
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position = 'topright',
    separator = 'Long:', 
    empty_string = 'NaN', 
    lng_first=False, 
    num_digits=20, 
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter, 

)
site_map.add_child(mouse_position)
display(site_map)

In [30]:
from math import sin, cos, sqrt, atan2, radians 

def calculate_distances(lat1, lon1, lat2, lon2): 
  # approximate radius of earth in km 
  R = 6373.0 

  lat1 = radians(lat1)
  lon1 = radians(lon1) 
  lat2 = radians(lat2) 
  lon2 = radians(lon2) 

  dlon = lon2 - lon1 
  dlat = lat2 - lat1 

  a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 
  c = 2 * atan2(sqrt(a), sqrt(1-a)) 

  distance = R * c 
  return distance

In [36]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline = calculate_distances(28.562302,	-80.577356, 28.56532, -80.56832)
print('The distance is:',distance_coastline)

The distance is: 0.9444149774521139


In [38]:
# Create and add a folium.Marker on your selected closest coastline point on the map

coastline_lat = 28.56367
coastline_lon = -80.56809
coastline_coordinate = [coastline_lat, coastline_lon]

# The distance_coastline variable was calculated in the cell above 

# Create a marker at the coastline coordinate
distance_marker = folium.Marker(
    coastline_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)

# Add the marker to the site map
site_map.add_child(distance_marker)

# Optionally, you could add a line connecting the launch site and the coastline point for visualization
# Get the launch site coordinates used in the previous cell (I88kMO1Xw_td)
# launch_site_lat = launch_sites_df.loc[0, 'Lat']
# launch_site_lon = launch_sites_df.loc[0, 'Long']
# launch_site_coordinate = [launch_site_lat, launch_site_lon]
# points = [launch_site_coordinate, coastline_coordinate]
# folium.PolyLine(points, color='blue', weight=2.5, opacity=1).add_to(site_map)


# Display the map to see the new marker
display(site_map)

In [40]:
# Get the coordinates for CCAFS SLC 40
launch_site_lat = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS SLC-40']['Lat'].iloc[0]
launch_site_lon = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS SLC-40']['Long'].iloc[0]
launch_site_coordinate = [launch_site_lat, launch_site_lon]

# Use the coastline coordinates you provided for CCAFS SLC 40
coastline_lat = 28.56532
coastline_lon = -80.56832
coastline_coordinate = [coastline_lat, coastline_lon]

# Create a list of the two coordinate pairs
points = [launch_site_coordinate, coastline_coordinate]

# Create a folium.PolyLine object
lines = folium.PolyLine(locations=points, weight=1, color='blue')

# Add the line to the site map
site_map.add_child(lines)

# Display the map
display(site_map)

In [42]:
# Calculate the distance to Orlando
launch_site_lat = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS SLC-40']['Lat'].iloc[0]
launch_site_lon = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS SLC-40']['Long'].iloc[0]
launch_site_coordinate = [launch_site_lat, launch_site_lon]

Orlando_lat = 28.54223
Orlando_lon = -81.37891
Orlando_coordinate = [Orlando_lat, Orlando_lon]

distance_Orlando = calculate_distances(launch_site_lat, launch_site_lon, Orlando_lat, Orlando_lon)

print(f'The distance to Orlando from CCAFS SLC-40 is: {distance_Orlando:.2f} km')


# Create a marker at the Orlando coordinate
distance_marker = folium.Marker(
    Orlando_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_Orlando),
    )
)

# Add the marker to the site map
site_map.add_child(distance_marker)

# Create a list of the two coordinate pairs for the line
points = [launch_site_coordinate, Orlando_coordinate]

# Create a folium.PolyLine object
lines = folium.PolyLine(locations=points, weight=2.5, color='blue')

# Add the line to the site map
site_map.add_child(lines)

# Display the map
display(site_map)

The distance to Orlando from CCAFS SLC-40 is: 78.40 km


In [44]:
distance_highway = calculate_distances(28.562302,	-80.577356, 28.5637, -80.57084)
print('The distance is:',distance_highway) 


highway_lat = 28.5637
highway_lon = -80.57084
highway_coordinate = [highway_lat, highway_lon]

# The distance_coastline variable was calculated in the cell above (I88kMO1Xw_td)

# Create a marker at the coastline coordinate
distance_marker = folium.Marker(
    highway_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
    )
)

# Add the marker to the site map
site_map.add_child(distance_marker)

# Optionally, you could add a line connecting the launch site and the coastline point for visualization
# Get the launch site coordinates used in the previous cell (I88kMO1Xw_td)
# launch_site_lat = launch_sites_df.loc[0, 'Lat']
# launch_site_lon = launch_sites_df.loc[0, 'Long']
# launch_site_coordinate = [launch_site_lat, launch_site_lon]
# points = [launch_site_coordinate, coastline_coordinate]
# folium.PolyLine(points, color='blue', weight=2.5, opacity=1).add_to(site_map)


# Display the map to see the new marker


# Get the coordinates for CCAFS SLC 40
launch_site_lat = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS SLC-40']['Lat'].iloc[0]
launch_site_lon = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS SLC-40']['Long'].iloc[0]
launch_site_coordinate = [launch_site_lat, launch_site_lon]

# Use the coastline coordinates you provided for CCAFS SLC 40
highway_lat = 28.5637
highway_lon = -80.57084
highway_coordinate = [highway_lat, highway_lon]

# Create a list of the two coordinate pairs
points = [launch_site_coordinate, highway_coordinate]

# Create a folium.PolyLine object
lines = folium.PolyLine(locations=points, weight=1, color='blue')

# Add the line to the site map
site_map.add_child(lines)

# Display the map
display(site_map)

The distance is: 0.6552802541545388


In [46]:
distance_rail = calculate_distances(28.562302,	-80.577356, 28.56324, -80.57714)
print('The distance is:',distance_rail) 


rail_line_lat = 28.56324
rail_line_lon = -80.57714
rail_line_coordinate = [rail_line_lat, rail_line_lon]

# The distance_coastline variable was calculated in the cell above (I88kMO1Xw_td)

# Create a marker at the coastline coordinate
distance_marker = folium.Marker(
    rail_line_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_rail),
    )
)

# Add the marker to the site map
site_map.add_child(distance_marker)

# Optionally, you could add a line connecting the launch site and the coastline point for visualization
# Get the launch site coordinates used in the previous cell (I88kMO1Xw_td)
# launch_site_lat = launch_sites_df.loc[0, 'Lat']
# launch_site_lon = launch_sites_df.loc[0, 'Long']
# launch_site_coordinate = [launch_site_lat, launch_site_lon]
# points = [launch_site_coordinate, coastline_coordinate]
# folium.PolyLine(points, color='blue', weight=2.5, opacity=1).add_to(site_map)


# Display the map to see the new marker


# Get the coordinates for CCAFS SLC 40
launch_site_lat = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS SLC-40']['Lat'].iloc[0]
launch_site_lon = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS SLC-40']['Long'].iloc[0]
launch_site_coordinate = [launch_site_lat, launch_site_lon]

# Use the coastline coordinates you provided for CCAFS SLC 40
rail_line_lat = 28.56324
rail_line_lon = -80.57714
rail_line_coordinate = [rail_line_lat, rail_line_lon]

# Create a list of the two coordinate pairs
points = [launch_site_coordinate, rail_line_coordinate]

# Create a folium.PolyLine object
lines = folium.PolyLine(locations=points, weight=1, color='blue')

# Add the line to the site map
site_map.add_child(lines)

# Display the map
display(site_map)

The distance is: 0.10644610430283738
